## 如何在调用工具时使用少样本提示

```{=mdx}
:::info 前提条件

本指南假定您熟悉以下概念：

- [聊天模型](/docs/concepts/chat_models)
- [LangChain 工具](/docs/concepts/tools)
- [工具调用](/docs/concepts/tool_calling)
- [将工具输出传递给聊天模型](/docs/how_to/tool_results_pass_to_model/)

:::
```

在更复杂的工具使用场景中，在提示中添加少量示例是非常有帮助的。我们可以通过在提示中添加包含 `ToolCalls` 的 `AIMessages` 以及相应的 `ToolMessages` 来实现这一点。

首先定义一个模型和一个计算器工具：

In [1]:
import { tool } from "@langchain/core/tools";
import { z } from "zod";
import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({ model: "gpt-4o", temperature: 0, })

/**
 * Note that the descriptions here are crucial, as they will be passed along
 * to the model along with the class name.
 */
const calculatorSchema = z.object({
  operation: z
    .enum(["add", "subtract", "multiply", "divide"])
    .describe("The type of operation to execute."),
  number1: z.number().describe("The first number to operate on."),
  number2: z.number().describe("The second number to operate on."),
});

const calculatorTool = tool(async ({ operation, number1, number2 }) => {
  // Functions must return strings
  if (operation === "add") {
    return `${number1 + number2}`;
  } else if (operation === "subtract") {
    return `${number1 - number2}`;
  } else if (operation === "multiply") {
    return `${number1 * number2}`;
  } else if (operation === "divide") {
    return `${number1 / number2}`;
  } else {
    throw new Error("Invalid operation.");
  }
}, {
  name: "calculator",
  description: "Can perform mathematical operations.",
  schema: calculatorSchema,
});

const llmWithTools = llm.bindTools([calculatorTool]);

我们的计算器可以处理常见的加法、减法、乘法和除法。但是，如果我们询问一个新的数学运算符 `🦜` 会出现什么情况呢？

让我们看看如果直接使用它会出现什么情况：

In [2]:
const res = await llmWithTools.invoke("What is 3 🦜 12");

console.log(res.content);
console.log(res.tool_calls);


[
  {
    name: 'calculator',
    args: { operation: 'multiply', number1: 3, number2: 12 },
    type: 'tool_call',
    id: 'call_I0oQGmdESpIgcf91ej30p9aR'
  }
]


它不太清楚如何将 `🦜` 解释为一个运算符，默认情况下会将其视为 `multiply`（乘法）。现在，我们尝试以构造消息的形式给它一些示例，引导它将 `🦜` 理解为 `divide`（除法）：

In [3]:
import { HumanMessage, AIMessage, ToolMessage } from "@langchain/core/messages";

const res = await llmWithTools.invoke([
  new HumanMessage("What is 333382 🦜 1932?"),
  new AIMessage({
    content: "The 🦜 operator is shorthand for division, so we call the divide tool.",
    tool_calls: [{
      id: "12345",
      name: "calculator",
      args: {
        number1: 333382,
        number2: 1932,
        operation: "divide",
      }
    }]
  }),
  new ToolMessage({
    tool_call_id: "12345",
    content: "The answer is 172.558."
  }),
  new AIMessage("The answer is 172.558."),
  new HumanMessage("What is 6 🦜 2?"),
  new AIMessage({
    content: "The 🦜 operator is shorthand for division, so we call the divide tool.",
    tool_calls: [{
      id: "54321",
      name: "calculator",
      args: {
        number1: 6,
        number2: 2,
        operation: "divide",
      }
    }]
  }),
  new ToolMessage({
    tool_call_id: "54321",
    content: "The answer is 3."
  }),
  new AIMessage("The answer is 3."),
  new HumanMessage("What is 3 🦜 12?")
]);

console.log(res.tool_calls);

[
  {
    name: 'calculator',
    args: { number1: 3, number2: 12, operation: 'divide' },
    type: 'tool_call',
    id: 'call_O6M4yDaA6s8oDqs2Zfl7TZAp'
  }
]


而且我们可以看到它现在以正确的方式将 `🦜` 等同于 `divide` 操作！

## 相关内容

- 流式传输 [工具调用](/docs/how_to/tool_streaming/)
- 将 [运行时值传递给工具](/docs/how_to/tool_runtime)
- 从模型中获取 [结构化输出](/docs/how_to/structured_output/)